In [1]:
from dotenv import load_dotenv
import os
from pprint import pprint

# trace langchain execution
os.environ["LANGCHAIN_TRACING"] = "false"

# load in keys from .env. Have in format API_KEY="".
load_dotenv()
API_KEY = os.getenv("API_KEY")
LOCATION = os.getenv("LOCATION")
GOOGLE_PROJECT_ID = os.getenv("GOOGLE_PROJECT_ID") # ie, confident-jackle-123456

In [2]:
from google.cloud import aiplatform

aiplatform.init(project=GOOGLE_PROJECT_ID, location="us-central1")

In [3]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain import LLMChain, PromptTemplate
from langchain.llms import vertexai
from vertexai.preview.language_models import TextGenerationModel
from langchain.agents import AgentType, initialize_agent

In [4]:
import json

from tools import GetBasicMenuTool, GetDetailedMenuTool, FindItemIdTool, CreateOrderTool

## overview
In order to maintain customer privacy and minimize the ability for this agent to leak information, it is not going to have any memory (in the prompt). It is still going to be able to retrieve and relay a user's order, but that will be information retrieved from an api using a tool. 

This saves alot of tokens being fed into the input prompt

In [5]:
from square.client import Client
from square.api.catalog_api import CatalogApi
from pydantic import BaseModel, Field, ConfigDict
from langchain.tools.base import BaseTool
from typing import Optional, Type
from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)
client = Client(
    square_version='2023-08-16',
    access_token=API_KEY,
    environment='sandbox')

In [6]:
tools = [GetBasicMenuTool(), GetDetailedMenuTool(), FindItemIdTool(), CreateOrderTool()]

In [7]:
agent = initialize_agent(
    tools,
    vertexai.VertexAI(temperature=0.3),
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

In [10]:
agent.run("What types of coffee are available?")



> Entering new AgentExecutor chain...
 Action:
```
{
  "action": "get_basic_menu_tool",
  "action_input": {}
}
```

Observation: ['Muffin', 'Coffee', 'Danish']
Thought: Action:
```
{
  "action": "get_detailed_menu_tool",
  "action_input": {}
}
```


Observation: ['Small, Blueberry Muffin', 'Small, Banana Chocolate Chip Muffin', 'Small, Radish Muffin', 'Medium, Blueberry Muffin', 'Medium, Banana Chocolate Chip Muffin', 'Medium, Radish Muffin', 'Large, Blueberry Muffin', 'Large, Banana Chocolate Chip Muffin', 'Large, Radish Muffin', 'Small Coffee', 'Medium Coffee', 'Large Coffee', 'Blueberry Danish', 'Banana Chocolate Chip Danish', 'Radish Danish']
Thought: Action:
```
{
  "action": "Final Answer",
  "action_input": "We have small, medium, and large coffee."
}
```

> Finished chain.


'We have small, medium, and large coffee.'

## Chirp
TODO: setup google chirp just for a simple demo